In [22]:
import sys
import os
import pandas as pd
import spacy
import ast

%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
data = pd.read_csv(PATH + '/skills_matcher/data/data_final_raw.csv')
data.head(2)

,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'..."
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': RICS, 'label': 'SKILL'}, {'entity'..."


In [24]:
new_data = data.drop_duplicates(subset ="position").reset_index()
new_data.to_csv(PATH + '/skills_matcher/data/data_final_raw_new.csv')

In [25]:
def extract_skills_auto(s):
    words_pattern = '[a-z]+'
    return re.findall(words_pattern, s, flags=re.IGNORECASE)

In [26]:
import re

#  new with non word char inside entities
def extract_entities_2(s):
    pattern = re.compile("'entity': [a-zA-Z]+\W*([a-zA-Z]|\s)*")
    res = re.finditer(pattern, s)
    return [ mo.group().split(",")[0].strip().lstrip("'entity': ") for mo in res if mo.group() != ',']

def extract_lables(s):
    pattern = re.compile("'label':\s*'\w+'")
    res = re.finditer(pattern, s)
    return [ mo.group().partition("': '")[2].strip("'") for mo in res]

In [27]:
def get_dict(x):
    
    dict_ = {}
    
    entities = extract_entities_2(x['entities_manual_label'])
    labels = extract_lables(x['entities_manual_label'])
    for index, key in enumerate(entities):
        if key not in dict_.keys():
            dict_[key] = labels[index]

    skills_auto =extract_skills_auto(x['entities_auto_label'])
    for index, value in enumerate(skills_auto):
        if value not in dict_.keys():
            dict_[value] = 'SKILL'
    
    skills_list = []
    knowledge_list = []
    min_exp_list = []
    level_list = []
            
    for entity, label in dict_.items():
        if label == 'SKILL':
            skills_list.append(entity)
        if label == 'KNOWLEDGE':
            knowledge_list.append(entity)
        if label == 'MIN_EXP':
             min_exp_list.append(entity)
        if label == 'LEVEL':
            level_list.append(entity)
    
    x["SKILL"] = skills_list
    x["KNOWLEDGE"] = knowledge_list
    x["MIN_EXP"] = min_exp_list
    x["LEVEL"] = level_list
    
    return x

In [28]:
data_cleaned = new_data.apply(get_dict, axis=1)

In [29]:
data_cleaned.head(2)

,index,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label,SKILL,KNOWLEDGE,MIN_EXP,LEVEL
0,0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'...","[English, German]",[physics],[],[]
1,3,3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...,"(Python, Python)","[{'entity': Python, 'label': 'SKILL'}, {'entit...","[Python, English, French]","[programming, High Energy Physics data analysi...",[],[]


In [30]:
def lowercase(s):
    new_list = [skill.lower() for skill in s]
    return new_list

def remove_punktuation(s):
    new_list = [re.sub(r'[^\w\s+]', '', skill).replace('\xa0', '') for skill in s]
    return new_list

In [31]:
data_cleaned['SKILL'] = data_cleaned['SKILL'].apply(lowercase).apply(remove_punktuation)
data_cleaned['KNOWLEDGE'] = data_cleaned['KNOWLEDGE'].apply(lowercase).apply(remove_punktuation)
data_cleaned['MIN_EXP'] = data_cleaned['MIN_EXP'].apply(lowercase).apply(remove_punktuation)
data_cleaned['LEVEL'] = data_cleaned['LEVEL'].apply(lowercase).apply(remove_punktuation)

In [37]:
def print_final_table(position, data):
    
    index = data.index[data['position'] == position].to_list()
    
    list_skills = data['SKILL'][index].to_list()[0]
    list_knowledge = data['KNOWLEDGE'][index].to_list()[0]
    list_min_exp = data['MIN_EXP'][index].to_list()[0]
    list_level = data['LEVEL'][index].to_list()[0]
    
    table = pd.DataFrame({'skills': pd.Series(list_skills), 
                         'knowledge': pd.Series(list_knowledge),
                         'level': pd.Series(list_min_exp),
                         'min_exp': pd.Series(list_level)})
    
    table = table.fillna('')
    
    return table

In [38]:
print_final_table('Applied Physicist (Computing) (EP-LBC-2021-125-LD) id54308', data_cleaned)

/var/folders/sy/7155ny0j7zd0yvnmgd8twpxm0000gn/T/ipykernel_64724/377553510.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'level': pd.Series(list_min_exp),
/var/folders/sy/7155ny0j7zd0yvnmgd8twpxm0000gn/T/ipykernel_64724/377553510.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'min_exp': pd.Series(list_level)})


,skills,knowledge,level,min_exp
0,python,programming,,
1,english,high energy physics data analysis,,
2,french,programming techniques,,


In [34]:
# sums = []
# for index, row in enumerate(data_cleaned['SKILL'] + data_cleaned['KNOWLEDGE'] + data_cleaned['MIN_EXP'] + data_cleaned['LEVEL']):
#     sums.append(len((data_cleaned['SKILL'] + data_cleaned['KNOWLEDGE'] + data_cleaned['MIN_EXP'] + data_cleaned['LEVEL'][index])))
#     
# data_cleaned['number_entities'] = sums

In [87]:
def clean_level(s):
    valid_level = ['phd','master','bachelor','hnd','hnc','degree']
    better_list = []
    for degree in s:
        better_list += degree.split(" ")
        
    new_list = [degree.replace('a ','') for degree in better_list]
    new_list = [degree.replace(' and ',' ') for degree in new_list]
    new_list = [degree for degree in new_list if len(degree) > 2]
    new_list = [degree.replace('mscbsc','msc bsc') for degree in new_list]
    new_list = [degree.replace('msc','master') for degree in new_list]
    new_list = [degree.replace('masters','master') for degree in new_list]
    new_list = [degree.replace('bachelors','bachelor') for degree in new_list]
    new_list = [degree.replace('bsc','bachelor') for degree in new_list]
    new_list = [degree.replace('ba','bachelor') if not 'bachelor' in degree else 'bachelor' for degree in new_list]
    
    
    new_list = [degree.replace('postgraduate degree','degree') for degree in new_list]
    new_list = [degree.replace('university','degree') for degree in new_list]
    new_list = [degree.replace('educated','degree') for degree in new_list]
    new_list = [degree.replace('deploma','degree') for degree in new_list]
    new_list = [degree.replace('graduate','degree') for degree in new_list]
    new_list = [degree.replace('nvq','degree') for degree in new_list]
    new_list = [degree.replace(degree,'degree') if 'degree' in degree else degree for degree in new_list]
    
    new_list = [degree for degree in new_list if degree in valid_level]
    
    new_list = list(set(new_list))
    
    if len(new_list) > 1:
        if 'degree' in new_list:
            new_list.remove('degree')
            
            
    new_list = sorted(new_list, key=lambda x: valid_level.index(x))
    
    return new_list


In [88]:
data_cleaned['LEVEL'].apply(clean_level).to_list()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['degree'],
 ['phd', 'master'],
 [],
 ['phd', 'master'],
 [],
 [],
 [],
 [],
 [],
 ['degree'],
 [],
 ['phd'],
 [],
 ['master'],
 ['phd'],
 ['master'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['phd', 'master'],
 ['phd'],
 [],
 [],
 ['phd'],
 ['degree'],
 [],
 ['phd'],
 [],
 ['degree'],
 ['degree'],
 ['phd'],
 [],
 ['phd'],
 [],
 ['phd', 'master'],
 ['phd'],
 ['phd'],
 [],
 ['phd', 'bachelor'],
 ['phd', 'bachelor'],
 [],
 [],
 ['phd'],
 [],
 ['phd', 'master'],
 ['phd'],
 ['master', 'bachelor'],
 ['master'],
 [],
 [],
 ['degree'],
 ['master'],
 ['bachelor'],
 [],
 ['bachelor'],
 ['bachelor'],
 ['phd'],
 ['bachelor'],
 [],
 ['phd'],
 ['master'],
 ['phd'],
 ['phd'],
 ['bachelor'],
 [],
 [],
 ['master'],
 [],
 [],
 [],
 ['phd', 'master'],
 ['degree'],
 [],
 ['degree'],
 [],
 [],
 [],
 ['phd', 'bachelor'],
 ['phd'],
 [],
 [],
 ['degree'],
 ['master', 'bachelor'],
 ['degree'],
 [],
 ['master', 'bachelor'],
 ['degree'],
 [],
 ['bachelor'],
 ['master'],
 ['de